## Convert all netCDF NCAR timestep files to Zarr (64,64,64), round-robin across time across FileDB nodes

In [9]:
!pip install "xarray[complete]"

  Using cached xarray-2023.1.0-py3-none-any.whl (973 kB)
  Using cached numpy-1.24.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached packaging-23.1-py3-none-any.whl (48 kB)
     |████████████████████████████████| 12.3 MB 10.2 MB/s eta 0:00:01B/s eta 0:00:01:01██████████▎| 12.1 MB 10.2 MB/s eta 0:00:01
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached Bottleneck-1.3.7-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (355 kB)
     |████████████████████████████████| 47 kB 3.1 MB/s  eta 0:00:01
  Using cached eccodes-1.5.2.tar.gz (2.3 MB)
  Using cached cftime-1.6.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached dask-2023.5.0-py3-none-any.whl (1.2 MB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
     |██████████████████████

In [1]:
import xarray as xr
import os
# from dask.diagnostics import ProgressBar
# import zarr # No compute image with both xarray and zarr

<font color="orange">Don't delete the CD cell!</font>

In [1]:
%cd /home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF

/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF


In [ ]:
folder_path = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF'

# Loop over files in the folder
for file_name in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, file_name)):
        # Perform operations on each file

        # https://github.com/pangeo-data/pangeo/issues/150
        # with dask.config.set(get=dask.threaded.get):
#         with ProgressBar():
        data_xr = xr.open_dataset(file_name)
        # Set chunk size
        chunk_size = {"nnx": 64, "nny": 64, "nnz": 64}

        data_xr.chunk(chunk_size)

        # Disable compression
        encoding = {variable_name: {'compressor': None, 'chunks': (64, 64, 64)} for variable_name in data_xr.variables}

        # overwrite if exists
        data_xr.to_zarr(store="/home/idies/workspace/turb/data02_02/" + file_name.replace(".", "-") + "-zarr-all",
                        mode="w",
                       encoding = encoding)
        
        print(file_name)
